In [1]:
!pip install pandas
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import pandasql as ps
import json

csv_name = 'cars.csv'

cars = pd.DataFrame(
    {
        'speed': [300, 200, 140],
        'engine': [5, 0, 3],
        'doors': [2, 4, 5],
    }, index=pd.Index(['lambo', 'tesla', 'lr'], name='car')
)

cars

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


In [2]:
cars.to_csv(csv_name)

In [3]:
cars = pd.read_csv(csv_name, index_col='car')
cars_wo_index = pd.read_csv(csv_name)

In [4]:
cars
cars_wo_index

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


,car,speed,engine,doors
0,lambo,300,5,2
1,tesla,200,0,4
2,lr,140,3,5


In [5]:
ti = pd.read_csv('titanic.csv', index_col=0)
ti

,Name,PClass,Age,Sex,Survived,SexCode
PassengerID,,,,,,
1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0
...,...,...,...,...,...,...
1309,"Zakarian, Mr Artun",3rd,27.00,male,0,0
1310,"Zakarian, Mr Maprieder",3rd,26.00,male,0,0
1311,"Zenni, Mr Philip",3rd,22.00,male,0,0


In [6]:
sql = """
select 
       --sex, survived,
       count(Name), count(Age), count(SexCode), count(*)
  from ti
--  group by sex, survived
--  order by sex, survived
"""
sql1 = """
select case when age is null then 'null' else 'not null' end as age_is_null, count(*)
  from ti
  group by case when age is null then 'null' else 'not null' end
"""

In [7]:
ps.sqldf(sql)
ps.sqldf(sql1)
ps.sqldf("select * from ti where age is null limit 5")
ps.sqldf("select * from ti where age is not null limit 5")

,count(Name),count(Age),count(SexCode),count(*)
0,1313,756,1313,1313


,age_is_null,count(*)
0,not null,756
1,null,557


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,13,"Aubert, Mrs Leontine Pauline",1st,None,female,1,1
1,14,"Barkworth, Mr Algernon H",1st,None,male,1,0
2,15,"Baumann, Mr John D",1st,None,male,0,0
3,30,"Borebank, Mr John James",1st,None,male,0,0
4,33,"Bradley, Mr George",1st,None,male,1,0


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
1,2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
2,3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
3,4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
4,5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0


In [8]:
ps.sqldf("select * from ti where PClass='*'")

,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,457,Jacobsohn Mr Samuel,*,None,male,0,0


In [9]:
ti.loc[ti.PClass=='*']

,Name,PClass,Age,Sex,Survived,SexCode
PassengerID,,,,,,
457,Jacobsohn Mr Samuel,*,NaN,male,0,0


In [10]:
ti.PClass=='*'

PassengerID
1       False
2       False
3       False
4       False
5       False
        ...  
1309    False
1310    False
1311    False
1312    False
1313    False
Name: PClass, Length: 1313, dtype: bool

In [11]:
sql_surv_class = """
select 
       PClass, survived, count(*)
  from ti
  group by PClass, survived
  order by 1, 2
"""

In [12]:
ps.sqldf(sql_surv_class)

,PClass,Survived,count(*)
0,*,0,1
1,1st,0,129
2,1st,1,193
3,2nd,0,160
4,2nd,1,119
5,3rd,0,573
6,3rd,1,138


In [13]:
ti.pivot_table(
    index=['Sex'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Sex,,,
female,38.0,28.0,22.0
male,42.0,28.0,25.5


In [14]:
ti.pivot_table(
    index=['Survived'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Survived,,,
0,46.0,29.5,25.0
1,36.0,25.0,22.0


In [15]:
data = pd.read_csv('weights.csv', index_col=0)
data.index.name = 'date'
data

,AT0000A18XM4 SW,BE0974268972 BB,US0527691069 US,DE0007164600 GR,US6092071058 US
date,,,,,
2014-01-14,0.184660,0.275706,0.303425,0.042841,0.193368
2014-01-15,0.225875,0.147861,0.151211,0.224074,0.250978
2014-01-16,0.261017,0.334213,0.293336,0.015744,0.095690
2014-01-17,0.160168,0.231034,0.092674,0.269050,0.247074
2014-01-18,0.057162,0.254146,0.027286,0.480660,0.180746
...,...,...,...,...,...
2018-02-28,0.123821,0.167899,0.120844,0.088780,0.498657
2018-03-01,0.035043,0.289624,0.220871,0.136366,0.318096
2018-03-02,0.397155,0.362025,0.050554,0.020247,0.170019


In [16]:
sql_data = """
select case
         when sum_of_columns = 1 then '==1'
         when sum_of_columns < 1 then '<1'
         when sum_of_columns > 1 then '>1'
         else '?'
       end as sum_rel_to_1,
       sum_of_columns, count(*) as qnty, min(date) as min_date, max(date) as max_date
  from (
    select date, "AT0000A18XM4 SW" + "BE0974268972 BB" + "US0527691069 US" + "DE0007164600 GR" + "US6092071058 US" as sum_of_columns
      from data
      --where sum_of_columns=1
  ) a
  group by sum_of_columns
  order by qnty desc
"""

# pd.set_option("display.precision", 40)
data_cols_sum = ps.sqldf(sql_data)
data_cols_sum.to_csv('weights_cols_summary.csv')
with pd.option_context('display.precision', 20, 'display.max_rows', 10):
    data_cols_sum

,sum_rel_to_1,sum_of_columns,qnty,min_date,max_date
0,==1,1.00000000000000000000,165,2014-01-15,2018-02-13
1,>1,1.00000000000009992007,46,2014-01-14,2018-01-04
2,>1,1.00000000000029998226,45,2014-01-21,2018-02-02
3,<1,0.99999999999979993781,44,2014-03-15,2018-01-02
4,>1,1.00000000000020006219,37,2014-02-19,2018-01-27
...,...,...,...,...,...
145,<1,0.99999999999900024417,1,2016-04-12,2016-04-12
146,<1,0.99999999999894995106,1,2017-11-23,2017-11-23
147,<1,0.99999999999892008606,1,2015-12-31,2015-12-31
148,<1,0.99999999999869992884,1,2017-02-23,2017-02-23


In [17]:
import numpy as np


def quantile_high(x):
    return x.quantile(.99, interpolation='nearest')


def quantile_low(x):
    return x.quantile(.01, interpolation='nearest')


def quantile_50(x):
    return x.quantile(.5, interpolation='nearest')


def second(x):
    return sorted(x.array)[1]


def before_last(x):
    return sorted(x.array)[-2]


with pd.option_context('display.precision', 20, 'display.max_rows', 20):
    data.agg([np.min, second, quantile_low, np.max, before_last, quantile_high, np.sum, np.median, quantile_50])

,AT0000A18XM4 SW,BE0974268972 BB,US0527691069 US,DE0007164600 GR,US6092071058 US
amin,0.00006324065623360000,0.00001772070035710000,0.00033072555740600002,0.00000028957406732000,0.00013168698548799999
second,0.00056832608789499995,0.00005853430333830000,0.00060501499727000005,0.00089959071450700003,0.00085935616687600003
quantile_low,0.00350278153427000015,0.00493473843680999977,0.00415517367883999969,0.00620237987144000007,0.00723891980141999972
amax,0.58878311877500000282,0.63858269868700001659,0.79298245820900004066,0.85554993405599999434,0.62190366982900002757
before_last,0.51422848409499999356,0.63294675020099999685,0.71303513476600000054,0.58111496260200001807,0.55771674779000002253
quantile_high,0.45644152804600002238,0.50502054067799995174,0.50583069728300000811,0.47684946836899999578,0.48791753556700001582
sum,207.61007571537618332513,219.56080655585805061492,221.51103382852187451135,217.21963889178613271724,215.09844500844010894980
median,0.18887609421900000695,0.20149625733699999541,0.20061813022700000775,0.19873642680199998800,0.20404436304299999883
quantile_50,0.18887609421900000695,0.20149625733699999541,0.20061813022700000775,0.19873642680199998800,0.20404436304299999883


In [18]:
s = pd.Series([2,5,1,4,3])
s.sort_values()
sorted(s.array)[-2]

2    1
0    2
4    3
3    4
1    5
dtype: int64

4

In [19]:
f_name = 'fines.json'
# , orient='values'
# fines = 
with open(f_name) as jf:
    j = json.load(jf)
print('json keys', j.keys())
for key in list(j.keys())[:-1]:
    print(f'    {key}({type(j[key])}): {j[key]}')
print()

jv = json.loads(j['Value'])
print('json Value keys', jv.keys())
print(f'    State', jv['State'])
print()

fines = jv['Fines']

print('Fines[0] key: value:')
for k, v in fines[0].items():
    print(f'    {k}({type(v)}): {v}')
print()

json keys dict_keys(['Message', 'ShowMessage', 'StateCode', 'TimeStamp', 'Value'])
    Message(<class 'str'>): Операция завершилась удачно!
    ShowMessage(<class 'bool'>): False
    StateCode(<class 'str'>): 0
    TimeStamp(<class 'str'>): 07.03.2018 15:28:44

json Value keys dict_keys(['Fines', 'State'])
    State 0

Fines[0] key: value:
    AdminOffensesTypeText(<class 'str'>): Штраф
    AmountToPay(<class 'int'>): 0
    ApnDetail(<class 'list'>): [{'AttrId': 'StAP', 'Name': 'Статья КоАП или закона субъекта РФ, состав правонарушения', 'Value': '12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге'}, {'AttrId': 'MestoDAP', 'Name': 'Место составления документа', 'Value': 'САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н'}, {'AttrId': 'DatNar', 'Name': 'Дата и время нарушения', 'Value': '23.01.2018 00:21:00'}, {'AttrId': 'MestoNar', 'Name': 'Место нарушения', 'Value': 'МОСКВА Г.   МОХОВАЯ д.15/1'}, {'AttrId': 'OdpsName', 'Name': 'Орган власти, выяви

In [20]:
import numpy as np
df = pd.DataFrame(data=[x.values() for x in fines], columns=fines[0].keys())
with pd.option_context("max_colwidth", 120):
    df1: pd.DataFrame = df[['AdminOffensesTypeText', 'AmountToPay', 'DAPName']]
    # df1
    # df1[[not x for x in np.isnan(df.AmountToPay)]]
    df1.loc[~np.isnan(df.AmountToPay)]

,AdminOffensesTypeText,AmountToPay,DAPName
0,Штраф,0.0,Постановление по видеофиксации №18810177180125209293 от 25.01.2018
258,Штраф,0.0,Постановление по видеофиксации №18810177180114012094 от 14.01.2018
545,Штраф,0.0,Постановление по видеофиксации №18810177180102236898 от 02.01.2018
817,Штраф,0.0,Протокол по делу 20.25 №0356043010117070700001076 от 07.07.2017


In [48]:
with pd.option_context("max_colwidth", 1000):
    df.loc[0][['ApnDetail']]
    df.head(3)

ApnDetail    [{'AttrId': 'StAP', 'Name': 'Статья КоАП или закона субъекта РФ, состав правонарушения', 'Value': '12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге'}, {'AttrId': 'MestoDAP', 'Name': 'Место составления документа', 'Value': 'САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н'}, {'AttrId': 'DatNar', 'Name': 'Дата и время нарушения', 'Value': '23.01.2018 00:21:00'}, {'AttrId': 'MestoNar', 'Name': 'Место нарушения', 'Value': 'МОСКВА Г.   МОХОВАЯ д.15/1'}, {'AttrId': 'OdpsName', 'Name': 'Орган власти, выявивший нарушение', 'Value': 'ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве'}, {'AttrId': 'FIONarush', 'Name': 'Нарушитель', 'Value': 'ЛЕОНИД ЭДУАРДОВИЧ'}, {'AttrId': 'GRZNarush', 'Name': 'Транспортное средство', 'Value': 'С700*****'}]
Name: 0, dtype: object

,AdminOffensesTypeText,AmountToPay,ApnDetail,DAP,DAPName,DateDAP,DiscountDate,DiscountSize,DiscountSizeAmountToPaySum,FineStatus,...,OdpsKod,PayedAmountSum,PaymentId,Sh,Sts,Type,UID,VidNak,VstupDate,Vu
0,Штраф,0.0,"[{'AttrId': 'StAP', 'Name': 'Статья КоАП или закона субъекта РФ, состав правонарушения', 'Value': '12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге'}, {'AttrId': 'MestoDAP', 'Name': 'Место составления документа', 'Value': 'САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н'}, {'AttrId': 'DatNar', 'Name': 'Дата и время нарушения', 'Value': '23.01.2018 00:21:00'}, {'AttrId': 'MestoNar', 'Name': 'Место нарушения', 'Value': 'МОСКВА Г. МОХОВАЯ д.15/1'}, {'AttrId': 'OdpsName', 'Name': 'Орган власти, выявивший нарушение', 'Value': 'ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве'}, {'AttrId': 'FIONarush', 'Name': 'Нарушитель', 'Value': 'ЛЕОНИД ЭДУАРДОВИЧ'}, {'AttrId': 'GRZNarush', 'Name': 'Транспортное средство', 'Value': 'С700*****'}]",18810177180125209293,Постановление по видеофиксации №18810177180125209293 от 25.01.2018,25.01.2018,,,None,1,...,1145519,None,,"{""DAP"":""18810177180125209293"",""DateDAP"":""25.01.2018"",""ODPS"":""ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве"",""PaymentID"":null,""RsRschet"":""40101810045250010041"",""SgBuk"":""044525000"",""ShBankName"":""ГУ Банка России по ЦФО"",""ShInn"":""7707089101"",""ShKbk"":""18811630020016000140"",""ShKpp"":""770731005"",""ShOkato"":""45379000"",""ShPoluchName"":""УФК по г. Москве (УГИБДД ГУ МВД России по г.Москве)"",""Sum"":3000,""Uin"":""18810177180125209293""}",7741525886,0,f9e58750-5f64-4138-942a-67e0967d4547,2,,
1,Штраф,NaN,"[{'AttrId': 'StAP', 'Name': 'Статья КоАП или закона субъекта РФ, состав правонарушения', 'Value': '12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч'}, {'AttrId': 'MestoDAP', 'Name': 'Место составления документа', 'Value': 'БОЛЬШИЕ ВЯЗЕМЫ РП,ЯМСКАЯ,д.1,стр.1,ОДИНЦОВСКИЙ Р-Н'}, {'AttrId': 'DatNar', 'Name': 'Дата и время нарушения', 'Value': '10.09.2017 10:31:00'}, {'AttrId': 'MestoNar', 'Name': 'Место нарушения', 'Value': ' А/Д М-9 ""БАЛТИЯ"", 24КМ+750М, ИЗ МОСКВЫ'}, {'AttrId': 'OdpsName', 'Name': 'Орган власти, выявивший нарушение', 'Value': 'Центр видеофиксации ГИБДД ГУ МВД России по Московской области'}, {'AttrId': 'FIONarush', 'Name': 'Нарушитель', 'Value': 'ЛЕОНИД ЭДУАРДОВИЧ'}, {'AttrId': 'GRZNarush', 'Name': 'Транспортное средство', 'Value': 'С700*****'}, {'AttrId': 'EffDate', 'Name': 'Дата вступления постановления в силу', 'Value': '27.10.2017'}]",18810150170911043896,Постановление по видеофиксации №18810150170911043896 от 11.09.2017,11.09.2017,,,None,1,...,1146511,None,,"{""DAP"":""18810150170911043896"",""DateDAP"":""11.09.2017"",""ODPS"":""Центр видеофиксации ГИБДД ГУ МВД России по Московской области"",""PaymentID"":null,""RsRschet"":""40101810845250010102"",""SgBuk"":""044525000"",""ShBankName"":""ГУ банка России по ЦФО"",""ShInn"":""7703037039"",""ShKbk"":""18811630020016000140"",""ShKpp"":""770245001"",""ShOkato"":""46641152"",""ShPoluchName"":""УФК по МО (УГИБДД ГУ МВД РОССИИ по МО)"",""Sum"":500,""Uin"":""18810150170911043896""}",7741525886,0,3e4c491f-df4e-4fad-99df-c08c3b4caef3,2,27.10.2017,
2,Штраф,NaN,"[{'AttrId': 'StAP', 'Name': 'Статья КоАП или закона субъекта РФ, состав правонарушения', 'Value': '12.9.6 - Повторное превышение скорости на 40-60 км/час'}, {'AttrId': 'MestoDAP', 'Name': 'Место составления документа', 'Value': 'САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н'}, {'AttrId': 'DatNar', 'Name': 'Дата и время нарушения', 'Value': '24.07.2017 10:19:00'}, {'AttrId': 'MestoNar', 'Name': 'Место нарушения', 'Value': 'МОСКВА Г. РУБЛЕВСКОЕ Ш. д.79'}, {'AttrId': 'OdpsName', 'Name': 'Орган власти, выявивший нарушение', 'Value': 'ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве'}, {'AttrId': 'FIONarush', 'Name': 'Нарушитель', 'Value': 'ЛЕОНИД ЭДУАРДОВИЧ'}, {'AttrId': 'GRZNarush', 'Name': 'Транспортное средство', 'Value': 'С700*****'}, {'AttrId': 'EffDate', 'Name': 'Дата вступления постановления 

In [54]:
print(10*'-')
for k, v in df.iterrows():
    if k > 10:
        print('End of max iterations reached.')
        break
    for attr_dict in v.ApnDetail:
        if attr_dict.get('AttrId') == 'StAP':
            print(f'{k}: {attr_dict.get("Value")}')

            print(10*'-')

----------
0: 12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге
1: 12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч
2: 12.9.6 - Повторное превышение скорости на 40-60 км/час
3: 12.15.1 - Наруш.правил расп.т/с на пр.части без выезда на ст.встр.движ.,дв.по обоч.,пересечение колонны
4: 12.9.6 - Повторное превышение скорости на 40-60 км/час
5: 12.15.1 - Наруш.правил расп.т/с на пр.части без выезда на ст.встр.движ.,дв.по обоч.,пересечение колонны
6: 12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч
7: 12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч
8: 12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч
9: 12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге
10: 12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге
End of max iterations reached.
----------


In [100]:
def find1(apn, key):
    res = None
    for ad in apn:
        if ad.get('AttrId') == key:
            res = ad.get('Value')
            break
    return res


index_name='id'
ind = []
names = []
df_dict = {'FineSum': []}
a_len = -1
for k, v in df.iterrows():
    a_len += 1
    ind.append(k)
    # 1st pass - names
    for attr_dict in v.ApnDetail:
        attr_id = attr_dict['AttrId']
        if attr_id not in names:
            names.append(attr_id)
            df_dict[attr_id] = [None for x in range(a_len)]
    # 2nd pass - values by names from 1st pass
    for attr_id in names:
        df_dict[attr_id].append(find1(v.ApnDetail, attr_id))
    df_dict['FineSum'].append(v.FineSum)

# for k, v in df_dict.items():
#     print(k, len(v))

df_apn = pd.DataFrame(df_dict, index=pd.Index(ind, name=index_name))
df_apn

,FineSum,StAP,MestoDAP,DatNar,MestoNar,OdpsName,FIONarush,GRZNarush,EffDate,FsspOtd,SentToFssp,SentToFsspDate,ClosedInFssp
id,,,,,,,,,,,,,
0,3000,12.17 ч.1.2 - Движение или остановка на полосе...,"САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н",23.01.2018 00:21:00,МОСКВА Г. МОХОВАЯ д.15/1,ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,None,None,None,None,None
1,500,12.09.2 - Превышение скорости движения ТС от 2...,"БОЛЬШИЕ ВЯЗЕМЫ РП,ЯМСКАЯ,д.1,стр.1,ОДИНЦОВСКИЙ...",10.09.2017 10:31:00,"А/Д М-9 ""БАЛТИЯ"", 24КМ+750М, ИЗ МОСКВЫ",Центр видеофиксации ГИБДД ГУ МВД России по Мос...,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,27.10.2017,None,None,None,None
2,2000,12.9.6 - Повторное превышение скорости на 40-6...,"САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н",24.07.2017 10:19:00,МОСКВА Г. РУБЛЕВСКОЕ Ш. д.79,ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,12.09.2017,None,None,None,None
3,1500,12.15.1 - Наруш.правил расп.т/с на пр.части бе...,"САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н",20.06.2017 00:07:00,МОСКВА Г. КУТУЗОВСКИЙ ПРОСП.,ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,05.08.2017,None,None,None,None
4,2000,12.9.6 - Повторное превышение скорости на 40-6...,"САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н",22.06.2017 19:47:00,МОСКВА Г. НОВЫЙ АРБАТ д.36/9,ЦАФАП ОДД ГИБДД ГУ МВД России по г.Москве,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,16.08.2017,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,500,12.09.2 - Превышение скорости движения ТС от 2...,"САДОВАЯ-САМОТЕЧНАЯ,д.1,ТВЕРСКОЙ (ЦАО) Р-Н",05.03.2018 22:10:00,МОСКВА Г. НОВЫЙ АРБАТ д.28,УГИБДД ГУ МВД России по г.Москве,ЛЕОНИД ЭДУАРДОВИЧ,С700*****,None,None,None,None,None
831,3000,"12.16.5 Нарушение, предусмотренное частью 4 на...","г. Москва, Тверская улица, 4",01.02.2018 16:28:43,"г. Москва, Тверская улица, 4",МАДИ,Леонид Эдуардович,С700*****,None,None,None,None,None
832,3000,"12.16.5 Нарушение, предусмотренное частью 4 на...","г. Москва, Тверская улица, 4",20.02.2018 11:44:39,"г. Москва, Тверская улица, 4",МАДИ,Леонид Эдуардович,С700*****,None,None,None,None,None


In [103]:
summary = df_apn.groupby('StAP')['FineSum'].agg(
    ['count', 'sum'],
).sort_values(['count'], ascending=False)

summary

,count,sum
StAP,,
12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч,361,180500
12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге,212,636000
"12.15.1 - Наруш.правил расп.т/с на пр.части без выезда на ст.встр.движ.,дв.по обоч.,пересечение колонны",95,142500
12.9.6 - Повторное превышение скорости на 40-60 км/час,94,188000
"12.15.5 - Повторный выезд на сторону, предназначенную для встречного движения",21,105000
12.9.7 - Повторное превышение скорости на 60-80 км/час и более 80 км/ч,17,85000
"12.16.5 Нарушение, предусмотренное частью 4 настоящей статьи, совершенное в городе федерального значения Москве или Санкт-Петербурге",15,45000
"12.16.1 - Несоблюдение требований знаков или разметки, за искл.случаев, предусм.др.статьями гл.12",8,4000
12.09.3 - Превышение скорости движения ТС от 40 до 60 км/ч,7,7000
